In [1]:
import pandas as pd
import scipy.stats as ss
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.api import anova_lm
from statsmodels.stats.multicomp import (pairwise_tukeyhsd, MultiComparison)
import pingouin as pg
import seaborn as sns

In [31]:
df = pd.read_csv('7_cars.csv')

In [48]:
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

In [5]:
df.describe()

,car_ID,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,103.000000,0.834146,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,3.329756,3.255415,10.142537,104.117073,5125.121951,25.219512,30.751220,13276.710571
std,59.322565,1.245307,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,0.270844,0.313597,3.972040,39.544167,476.985643,6.542142,6.886443,7988.852332
min,1.000000,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,13.000000,16.000000,5118.000000
25%,52.000000,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,3.150000,3.110000,8.600000,70.000000,4800.000000,19.000000,25.000000,7788.000000
50%,103.000000,1.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,3.310000,3.290000,9.000000,95.000000,5200.000000,24.000000,30.000000,10295.000000
75%,154.000000,2.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,3.580000,3.410000,9.400000,116.000000,5500.000000,30.000000,34.000000,16503.000000
max,205.000000,3.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,3.940000,4.170000,23.000000,288.000000,6600.000000,49.000000,54.000000,45400.000000


Создайте новый признак – марку автомобиля (company). Машины каких производителей встречаются в датасете? Далее исправьте названия и проверьте изменения.

In [ ]:
# мой вариант извлечения названия компании из названия автомобиля (слишком сложно)
df['company'] = df["CarName"].str.extract(r'^([^ ]+)')

In [32]:
df['company'] = df['CarName'].str.split(' ').str[0]

In [ ]:
# исправление опечаток в названиях компаний автомобилей ручным методом
df['company'] = df['company'].replace({'maxda': 'mazda', 'Nissan': 'nissan', 'porcshce': 'porsche', 'toyouta': 'toyota', 'vokswagen': 'volkswagen', 'vw': 'volkswagen'})

In [33]:
df['company'] = df['company'].str.lower()

In [35]:
def fix_company(to_rep, how_rep):
    df['company'] = df['company'].str.replace(to_rep, how_rep)

In [36]:
fix_company('maxda', 'mazda')
fix_company('porcshce', 'porsche')
fix_company('toyouta', 'toyota')
fix_company('vokswagen', 'volkswagen')
fix_company('vw', 'volkswagen')

In [37]:
df['company'].nunique()

22

Оставьте только часть предикторов, после чего посчитайте корреляцию между price и другими переменными

In [38]:
cols_to_keep = ['company', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'enginetype', 'cylindernumber', 'carlength', 'carwidth', 'curbweight', 'enginesize', 'wheelbase', 'boreratio', 'horsepower', 'price']

In [39]:
cars = df[cols_to_keep]
cars.head()

,company,fueltype,aspiration,carbody,drivewheel,enginetype,cylindernumber,carlength,carwidth,curbweight,enginesize,wheelbase,boreratio,horsepower,price
0,alfa-romero,gas,std,convertible,rwd,dohc,four,168.8,64.1,2548,130,88.6,3.47,111,13495.0
1,alfa-romero,gas,std,convertible,rwd,dohc,four,168.8,64.1,2548,130,88.6,3.47,111,16500.0
2,alfa-romero,gas,std,hatchback,rwd,ohcv,six,171.2,65.5,2823,152,94.5,2.68,154,16500.0
3,audi,gas,std,sedan,fwd,ohc,four,176.6,66.2,2337,109,99.8,3.19,102,13950.0
4,audi,gas,std,sedan,4wd,ohc,five,176.6,66.4,2824,136,99.4,3.19,115,17450.0


In [40]:
cars.select_dtypes(include='number').corr()
# корреляционная матрица

,carlength,carwidth,curbweight,enginesize,wheelbase,boreratio,horsepower,price
carlength,1.000000,0.841118,0.877728,0.683360,0.874587,0.606454,0.552623,0.682920
carwidth,0.841118,1.000000,0.867032,0.735433,0.795144,0.559150,0.640732,0.759325
curbweight,0.877728,0.867032,1.000000,0.850594,0.776386,0.648480,0.750739,0.835305
enginesize,0.683360,0.735433,0.850594,1.000000,0.569329,0.583774,0.809769,0.874145
wheelbase,0.874587,0.795144,0.776386,0.569329,1.000000,0.488750,0.353294,0.577816
boreratio,0.606454,0.559150,0.648480,0.583774,0.488750,1.000000,0.573677,0.553173
horsepower,0.552623,0.640732,0.750739,0.809769,0.353294,0.573677,1.000000,0.808139
price,0.682920,0.759325,0.835305,0.874145,0.577816,0.553173,0.808139,1.000000


In [41]:
corr_mat = cars.select_dtypes(include='number').corr()
corr_mat['price'].sort_values(ascending=False).round(2)
# корреляция признаков с колонкой price

price         1.00
enginesize    0.87
curbweight    0.84
horsepower    0.81
carwidth      0.76
carlength     0.68
wheelbase     0.58
boreratio     0.55
Name: price, dtype: float64

Последний шаг в подготовке данных: линейная регрессия в python не справляется с категориальными переменными (типом object в pandas), поэтому давайте применим функцию под названием pd.get dummies(). Она создаёт т.н. фиктивные переменные на основе изначальных категорий, представленные в виде 0 и 1. Официально такое преобразование называется One-Hot Encoding.  
Для начала примените её только на категориальных переменных:  
df_dummy = pd.get_dummies(data=cars[[список_столбцов_типа_object]], drop_first = True)  
Теперь примените её ко всему датафрейму (с тем же набором колонок, что и на предыдущем шаге). Не забудьте сохранить результат, он нам ещё понадобится :)

In [42]:
cars.dtypes

company            object
fueltype           object
aspiration         object
carbody            object
drivewheel         object
enginetype         object
cylindernumber     object
carlength         float64
carwidth          float64
curbweight          int64
enginesize          int64
wheelbase         float64
boreratio         float64
horsepower          int64
price             float64
dtype: object

In [ ]:
# код из ответов - неверный, т.к. создает дамми-переменные в виде булевых значений
cars_dummy = pd.get_dummies(data=cars[['company', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'enginetype', 'cylindernumber']], drop_first=True)
cars_dummy.head()
# создание дамми-переменных

,company_audi,company_bmw,company_buick,company_chevrolet,company_dodge,company_honda,company_isuzu,company_jaguar,company_mazda,company_mercury,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,True,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False


In [45]:
cars_dummy = pd.get_dummies(
    data=cars[['company', 'fueltype', 'aspiration', 'carbody',
               'drivewheel', 'enginetype', 'cylindernumber']],
    drop_first=True
).astype(int)

In [ ]:
# код из ответов - неверный, т.к. создает дамми-переменные в виде булевых значений
cars_lr = pd.get_dummies(data=cars, drop_first=True)
cars_lr.head()
# объединение всех признаков в одну таблицу для линейной регрессии

,carlength,carwidth,curbweight,enginesize,wheelbase,boreratio,horsepower,price,company_audi,company_bmw,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,168.8,64.1,2548,130,88.6,3.47,111,13495.0,False,False,...,False,False,False,False,False,True,False,False,False,False
1,168.8,64.1,2548,130,88.6,3.47,111,16500.0,False,False,...,False,False,False,False,False,True,False,False,False,False
2,171.2,65.5,2823,152,94.5,2.68,154,16500.0,False,False,...,False,False,True,False,False,False,True,False,False,False
3,176.6,66.2,2337,109,99.8,3.19,102,13950.0,True,False,...,True,False,False,False,False,True,False,False,False,False
4,176.6,66.4,2824,136,99.4,3.19,115,17450.0,True,False,...,True,False,False,False,True,False,False,False,False,False


In [ ]:
bool_cols = cars_lr.select_dtypes(include='bool').columns
cars_lr[bool_cols] = cars_lr[bool_cols].astype(int)
cars_lr.head()
# преобразование булевых признаков в числовые


,carlength,carwidth,curbweight,enginesize,wheelbase,boreratio,horsepower,price,company_audi,company_bmw,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,168.8,64.1,2548,130,88.6,3.47,111,13495.0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,168.8,64.1,2548,130,88.6,3.47,111,16500.0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,171.2,65.5,2823,152,94.5,2.68,154,16500.0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,176.6,66.2,2337,109,99.8,3.19,102,13950.0,1,0,...,1,0,0,0,0,1,0,0,0,0
4,176.6,66.4,2824,136,99.4,3.19,115,17450.0,1,0,...,1,0,0,0,1,0,0,0,0,0


Построим модель с одним предиктором цены (price) - horsepower. Какой процент изменчивости объясняет полученная модель?

In [53]:
results = smf.ols('price ~ horsepower', data=cars_lr).fit()
print(results.summary())
# линейная регрессия price от horsepower
# R² = 0.653, p-value << 0.05, коэффициент при horsepower положительный, значит, с ростом horsepower растет и price
# но модель объясняет только 65.3% дисперсии price
# intercept - это значение price при horsepower = 0
# intercept отрицательный, что означает, что при horsepower = 0 цена будет отрицательной, что нелогично и не интерпретируемо

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                 Пт, 26 дек 2025   Prob (F-statistic):           1.48e-48
Time:                        22:01:48   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3721.7615    929.849     -4.003      0.0

In [54]:
# сделаем intercept интерпретируемым, центрировав признак horsepower относительно его среднего значения
cars_lr['horsepower_centered'] = cars_lr['horsepower'] - cars_lr['horsepower'].mean()
results_cent = smf.ols('price ~ horsepower_centered', data=cars_lr).fit()
print(results_cent.summary())
# intercept = 13207.1293 - это цена автомобиля со средним horsepower
# коэффициент при horsepower_centered остался таким же, как и при horsepower

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                 Пт, 26 дек 2025   Prob (F-statistic):           1.48e-48
Time:                        22:01:54   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept            1.328e+04    

In [55]:
cars_lr['horsepower'].mean()
# среднее значение horsepower
# horsepower_centered = 163.2631 это означает что при увеличении horsepower на 1 единицу, цена автомобиля увеличивается в среднем на 163.2631 долларов

np.float64(104.1170731707317)

In [56]:
cars_lr['horsepower_centered'].round(3).head()

0     6.883
1     6.883
2    49.883
3    -2.117
4    10.883
Name: horsepower_centered, dtype: float64

In [57]:
cars_lr['horsepower_centered'].mean()
# среднее значение центрированного horsepower равно нулю

np.float64(5.47637815854126e-15)

In [58]:
cars_lr.drop(columns=['horsepower_centered'], inplace=True, axis=1)

Построим еще две модели:
- модель со всеми предикторами
- модель со всеми предикторами кроме марок машин
Выберем из этих моделей самую удачную. При выборе моделей нужно ориентироваться не только на количественные показатели, но также на контекст ее использования и здравый смысл.

In [59]:
x = cars_lr.drop(columns=['price'], axis='columns') # предикторы
y = cars_lr['price'] # целевая переменная
x = sm.add_constant(x) # добавление константы для расчета intercept
x

,const,carlength,carwidth,curbweight,enginesize,wheelbase,boreratio,horsepower,company_audi,company_bmw,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,1.0,168.8,64.1,2548,130,88.6,3.47,111,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1.0,168.8,64.1,2548,130,88.6,3.47,111,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1.0,171.2,65.5,2823,152,94.5,2.68,154,0,0,...,0,0,1,0,0,0,1,0,0,0
3,1.0,176.6,66.2,2337,109,99.8,3.19,102,1,0,...,1,0,0,0,0,1,0,0,0,0
4,1.0,176.6,66.4,2824,136,99.4,3.19,115,1,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,1.0,188.8,68.9,2952,141,109.1,3.78,114,0,0,...,1,0,0,0,0,1,0,0,0,0
201,1.0,188.8,68.8,3049,141,109.1,3.78,160,0,0,...,1,0,0,0,0,1,0,0,0,0
202,1.0,188.8,68.9,3012,173,109.1,3.58,134,0,0,...,0,0,1,0,0,0,1,0,0,0
203,1.0,188.8,68.9,3217,145,109.1,3.01,106,0,0,...,1,0,0,0,0,0,1,0,0,0


In [ ]:
model_1 = sm.OLS(y, x).fit()
print(model_1.summary())
# т.к. ранее мы выкинули одну марку компании, то теперь coef это отклонение от средней цены этой марки

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     81.09
Date:                 Пт, 26 дек 2025   Prob (F-statistic):           4.86e-89
Time:                        22:02:16   Log-Likelihood:                -1804.2
No. Observations:                 205   AIC:                             3702.
Df Residuals:                     158   BIC:                             3858.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -3.472e+

In [61]:
# удалим ненуное
cols_to_keep = cars_lr.columns[~cars_lr.columns.str.startswith('company_')]
cols_to_keep

Index(['carlength', 'carwidth', 'curbweight', 'enginesize', 'wheelbase',
       'boreratio', 'horsepower', 'price', 'fueltype_gas', 'aspiration_turbo',
       'carbody_hardtop', 'carbody_hatchback', 'carbody_sedan',
       'carbody_wagon', 'drivewheel_fwd', 'drivewheel_rwd', 'enginetype_dohcv',
       'enginetype_l', 'enginetype_ohc', 'enginetype_ohcf', 'enginetype_ohcv',
       'enginetype_rotor', 'cylindernumber_five', 'cylindernumber_four',
       'cylindernumber_six', 'cylindernumber_three', 'cylindernumber_twelve',
       'cylindernumber_two'],
      dtype='object')

In [62]:
# без марок
x_2 = cars_lr[cols_to_keep].drop(columns=['price'], axis='columns') # предикторы без марок
y_2 = cars_lr['price'] # целевая переменная 

In [63]:
x_2 = sm.add_constant(x_2) # добавление константы для расчета intercept

In [64]:
model_2 = sm.OLS(y_2, x_2).fit()
print(model_2.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     72.32
Date:                 Пт, 26 дек 2025   Prob (F-statistic):           9.86e-81
Time:                        22:19:59   Log-Likelihood:                -1881.6
No. Observations:                 205   AIC:                             3817.
Df Residuals:                     178   BIC:                             3907.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   -1.7e+

In [ ]:
# после того как мы убрали марки компаний, R² упал с 0.937 до 0.828, что говорит о том, что марки компаний вносят вклад в объяснение дисперсии цены автомобиля но не настолько значительный, 
# чтобы их обязательно включать в модель.
# если судить чисто по диагностическим показателям, типа R², то модель со всеми предикторами лучше, но с точки зрения интерпретируемости модель без марок компаний предпочтительнее.